# Innovation Center Booth Analyser
This is a project created by the innovation center team at Monshaat as a demo to the AI capabilities as one of the rising technologies the center focuses on.


In [ ]:
!pip install ultralytics
!pip install opencv-python

## Camera Test

In [1]:
import cv2

# Function to check available cameras
def check_cameras():
    index = 0
    arr = []
    while True:
        cap = cv2.VideoCapture(index)
        cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))

        if not cap.read()[0]:
            cap.release()  # Ensure the camera is released
            break
        else:
            arr.append(index)
        cap.release()
        index += 1
    return arr

available_cameras = check_cameras()
print(f"Available cameras: {available_cameras}")

Available cameras: []


[ WARN:0@0.035] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@0.037] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [3]:

import cv2
import time

# Open the first available camera (video0)
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))


if not cap.isOpened():
    print("Cannot open camera")
else:
    print("Camera opened successfully")
    start_time = time.time()
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        # Display the resulting frame
        cv2.imshow('Camera Test', frame)
        if cv2.waitKey(1) == ord('q') or (time.time() - start_time) > 20:
            break

    # Release the capture
    cap.release()
    cv2.destroyAllWindows()

Camera opened successfully


# Functions

### Get Camera

In [4]:
import cv2
def getCamera(id):
    cap = cv2.VideoCapture(id)
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
    return cap

def getVideo(path):
    cap = cv2.VideoCapture(path)
    assert cap.isOpened(), "Cannot open video"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(f'output_{extract_filename(path)}.avi', cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    return cap, video_writer

def extract_filename(path):
    # extract the filename without the path and extension
    return path.split('/')[-1].split('.')[0]

In [12]:
from ultralytics import solutions, YOLO

yolo = YOLO('yolo11n')
model = "yolo11n.pt"

from heatmapper import Heatmap
heatmap = Heatmap(colormap=cv2.COLORMAP_TURBO, show=False, model=model)



Ultralytics Solutions: ✅ {'model': 'yolo11n.pt', 'region': None, 'line_width': 2, 'show': False, 'show_in': True, 'show_out': True, 'classes': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'colormap': 20}


In [7]:

# region box from 90,200 to 200,400    
region = [(400,400),(1200,400)]

counter = solutions.ObjectCounter(
    model='yolo11n.pt',
    show=False,
    region=region,
)


Ultralytics Solutions: ✅ {'model': 'yolo11n.pt', 'region': [(400, 400), (1200, 400)], 'line_width': 2, 'show': False, 'show_in': True, 'show_out': True, 'classes': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'colormap': None}


In [8]:

# region box from 90,200 to 200,400    
region = [(400,500),(400,0)]

counter = solutions.ObjectCounter(
    model='yolo11n.pt',
    show=False,
    region=region,
)


Ultralytics Solutions: ✅ {'model': 'yolo11n.pt', 'region': [(400, 500), (400, 0)], 'line_width': 2, 'show': False, 'show_in': True, 'show_out': True, 'classes': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'colormap': None}


In [14]:
###
cap = getCamera(0)

i = 0
while cap.isOpened():
    
    success, im0 = cap.read()
    if not success:
        break

    # track objects in the frame
    results = yolo.track(im0)
    annotated_img = results[0].plot()

    # generate heatmap every 10 frames
    if i % 5 == 0:
        heatmapImg = heatmap.generate_heatmap(im0)
    
    annotated_img = counter.count(im0)
    # merge the heatmap with the original image
    im1 = cv2.addWeighted(annotated_img, 1, heatmapImg, 0.8, 0)
    
    cv2.imshow("Heatmap", im1)

    if cv2.waitKey(1) == ord('q'):
        break
    i += 1

cap.release()
cv2.destroyAllWindows()

[ WARN:0@65823.888] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@65823.923] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [ ]:
cap, video_writer = getVideo("v.mp4")

annotated_img = None
heatmapImg = None  

i = 0
while cap.isOpened():
    
    success, im0 = cap.read()
    if not success:
        break


##
    # track objects in the frame
    results = yolo.track(im0)
    annotated_img = results[0].plot()

    # generate heatmap every 10 frames
    if i % 10 == 0:
        heatmapImg = heatmap.generate_heatmap(im0)
    
    annotated_img = counter.count(im0)
    # merge the heatmap with the original image
    im1 = cv2.addWeighted(annotated_img, 1, heatmapImg, 0.5, 0)
    
##

    video_writer.write(im1)

    if cv2.waitKey(1) == ord('q') or i > 200:
        break
    
    i += 1

cap.release()
cv2.destroyAllWindows()

In [18]:
import cv2

# Open the default camera
cam = cv2.VideoCapture(0)

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

while True:
    ret, frame = cam.read()

    # Write the frame to the output file
    out.write(frame)

    # Display the captured frame
    cv2.imshow('Camera', frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break

# Release the capture and writer objects
cam.release()
out.release()
cv2.destroyAllWindows()

[ WARN:0@27535.533] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@27535.533] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


error: OpenCV(4.10.0) /io/opencv/modules/highgui/src/window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
